In [1]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import math

In [2]:
import data_file
male_list = data_file.male_list
female_list = data_file.female_list
male_professions = data_file.female_professions

In [4]:
male_list

[['laborer',
  'courier',
  'repairer',
  'cleaner',
  'conductor',
  'programmer',
  'chef',
  'helper',
  'estimator',
  'porter'],
 ['pipelayer',
  'mason',
  'conductor',
  'installer',
  'courier',
  'cleaner',
  'announcer',
  'electrician',
  'helper',
  'programmer'],
 ['cabinetmaker',
  'carpenter',
  'porter',
  'chef',
  'engineer',
  'conductor',
  'machinist',
  'brickmason',
  'pilot',
  'pipelayer'],
 ['painter',
  'courier',
  'drafter',
  'cabinetmaker',
  'programmer',
  'architect',
  'laborer',
  'dishwasher',
  'cleaner',
  'mason'],
 ['painter',
  'dishwasher',
  'announcer',
  'architect',
  'engineer',
  'repairer',
  'brickmason',
  'machinist',
  'chef',
  'courier'],
 ['mason',
  'drafter',
  'clergy',
  'electrician',
  'laborer',
  'cleaner',
  'roofer',
  'carpenter',
  'porter',
  'engineer'],
 ['brickmason',
  'laborer',
  'pipelayer',
  'installer',
  'painter',
  'carpenter',
  'engineer',
  'estimator',
  'clergy',
  'dishwasher'],
 ['architect',
  'm

In [4]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# # import dataset
# jobs_df =pd.DataFrame(["technician", "accountant", "supervisor", "engineer", "worker", "educator", "clerk", "counselor", "inspector", "mechanic", "manager", "therapist", "administrator", "salesperson", "receptionist", "librarian", "advisor", "pharmacist", "janitor", "psychologist", "physician", "carpenter", "nurse", "investigator", "bartender", "specialist", "electrician", "officer", "pathologist", "teacher", "lawyer", "planner", "practitioner", "plumber", "instructor", "surgeon", "veterinarian", "paramedic", "examiner", "chemist", "machinist", "appraiser", "nutritionist", "architect", "hairdresser", "baker", "programmer", "paralegal", "hygienist", "scientist"])
# job_list = jobs_df[0]
# job_list

In [6]:
# # gendered_word_list
# gendered_word_list = [["she", "he"],["female", "male"], ["woman", "man"], ["girl", "boy"], ["sister", "brother"], ["daughter", "son"], ["mother", "father"]]
XX = ["female", "woman", "girl", "sister", "daughter", "mother", "aunt", "grandmother", "daughter"]
YY = ["male", "man", "boy", "brother", "son", "father", "uncle", "grandfather", "son"]

In [7]:
templates = ["this is the _.", "that is the _.", "there is the _.", "here is the _.", "the _ is here.", "the _ is there.", "the _ is a person.", "it is the _.","the _ is a [MASK].", "the _ is an engineer.", "the _ is a nurse with superior technical skills."]

In [8]:
temps = ["temp1", "temp2", "temp3", "temp4","temp5", "temp6", "temp7", "temp8", "temp9", "temp10", "temp11"]

In [9]:
len(templates)

11

In [10]:
# CLS embedding
def sentence_embedding(template, word):
    sentence = template.replace("_", word)
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    token_embeddings = last_hidden_states
    return token_embeddings[0][0].cpu().detach().numpy()

In [11]:
def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

In [12]:
def construct_cossim_lookup(XY, AB):
    """
    XY: mapping from target string to target vector (either in X or Y)
    AB: mapping from attribute string to attribute vectore (either in A or B)
    Returns an array of size (len(XY), len(AB)) containing cosine similarities
    between items in XY and items in AB.
    """

    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


In [13]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [14]:

def s_XAB(A, s_wAB_memo):
    return s_wAB_memo[A].sum()

In [15]:
def s_wAB(X, Y, cossims):
    """
    Return vector of s(w, A, B) across w, where
        s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
    """
#     print((cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)).shape)
    return cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)

In [16]:
# def s_wAB_df(X, Y, cossims):
#     """
#     Return vector of s(w, A, B) across w, where
#         s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
#     """
# #     print(cossims[:, A].mean(axis=1))
#     df1 = pd.DataFrame(cossims[X, :].mean(axis=0))
#     df2 = pd.DataFrame(cossims[Y, :].mean(axis=0))
#     return df1, df2


def s_XAB_df(A, B, s_wAB_memo):
    df1 = pd.DataFrame(s_wAB_memo[A])
    df2 = pd.DataFrame(s_wAB_memo[B])
    return df1, df2

In [17]:
def s_XYAB(A, B, s_wAB_memo):
    r"""
    Given indices of target concept X and precomputed s_wAB values,
    the WEAT test statistic for p-value computation.
    """
    return s_XAB(A, s_wAB_memo) - s_XAB(B, s_wAB_memo)

In [18]:
# def WEAT_test(X, Y, A, B, n_samples, cossims, parametric=False):
def WEAT_test(X, Y, A, B, n_samples, cossims):
    ''' Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
    '''
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(X, Y, cossims=cossims)
#     print(s_wAB_memo)
    XY = np.concatenate((X, Y))

#     if parametric:
#     log.info('Using parametric test')
    s = s_XYAB(A, B, s_wAB_memo)
    return s


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [19]:
def get_effect_size(df1, df2, k=0):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [20]:
template_score_dict = {}
for ind, template in enumerate(templates):
    score_dict = {}
    attribute_template = template
    target_template = template
    X = {"x" + str(j): sentence_embedding(attribute_template, j) for j in XX}
    Y = {"y" + str(j): sentence_embedding(attribute_template, j) for j in YY}
    (X, Y) = convert_keys_to_ints(X, Y)
    XY = X.copy()
    XY.update(Y)
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    for i in range(len(female_list)):
        AA = female_list[i]
    #     print(AA)
    #     print(XX)
        BB = male_list[i]

        A = {"a" + str(j): sentence_embedding(target_template, j) for j in AA}
        B = {"b" + str(j): sentence_embedding(target_template, j) for j in BB}

        (A, B) = convert_keys_to_ints(A, B)


        AB = A.copy()
        AB.update(B)

        cossims = construct_cossim_lookup(XY, AB)
        A = np.array(list(A), dtype=np.int)
        B = np.array(list(B), dtype=np.int)


        s_wAB_memo = s_wAB(X, Y, cossims=cossims)
        df1,df2 = s_XAB_df(A, B, s_wAB_memo)
        effect_size = get_effect_size(df1, df2)
        score_dict[i] = effect_size
        print(score_dict[i])
    template_score_dict[temps[ind]] = score_dict
# log.info("pval: %g", pval)

# log.info("computing effect size...")
# esize = effect_size(X, Y, A, B, cossims=cossims)
# log.info("esize: %g", esize)

1.2183888162528902
1.149415303462839
1.087691148849411
1.177525663828664
1.4163164173814078
1.2950823985971622
1.26394475144944
1.0514220372148693
1.2145630053863175
1.2051310668137427
1.084822541111828
1.0279503178108953
1.236813052322568
0.8492021774860143
1.2921773761447448
1.1385837018024667
1.1856982827980402
1.0455117791192958
1.2339127500000027
1.2185111464976937
1.0853378066411714
0.9010622604331263
1.2870922616015028
1.2109165765618517
1.3551520146823806
1.32929939019638
1.1852843109101585
1.1686353477985958
0.8076340914485503
1.093693414899261
1.051167217016259
1.0027930103512999
1.2342107498476742
0.8747045058130719
1.3336289351988424
1.0206366972396446
1.0606982209372366
1.043784223645748
1.005573918413763
1.1309065010819188
1.1669763766530812
0.4181680738317021
0.4233535657053087
1.363037470444357
0.8739863751143749
0.6564722356270911
0.4567167840764625
0.5309601570159286
0.35611418144969714
0.5420311359164848
-0.09242134630229705
0.6543287714140602
0.7533059750563742
0.60

In [21]:
import json
a_file = open("seat_CLS_templates.json", "w")
json.dump(template_score_dict, a_file)
a_file.close()